<a href="https://colab.research.google.com/github/Rodrigo-Perico/DataScience/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import sklearn
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

In [38]:
import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("renatosn/sao-paulo-housing-prices")

files = os.listdir(path)
print("Arquivos no dataset:", files)

csv_file = [f for f in files if f.endswith(".csv")][0]
csv_path = os.path.join(path, csv_file)

df = pd.read_csv(csv_path)

print(df.head())


Arquivos no dataset: ['data.csv']
              address      district  area  bedrooms  garage  \
0          Rua Herval    Belenzinho    21         1       0   
1  Avenida São Miguel  Vila Marieta    15         1       1   
2    Rua Oscar Freire     Pinheiros    18         1       0   
3    Rua Júlio Sayago       Vila Ré    56         2       2   
4  Rua Barata Ribeiro    Bela Vista    19         1       0   

                 type  rent  total  
0     Studio e kitnet  2400   2939  
1     Studio e kitnet  1030   1345  
2         Apartamento  4000   4661  
3  Casa em condomínio  1750   1954  
4     Studio e kitnet  4000   4654  


In [39]:
ordinal_encoder = OrdinalEncoder()
adrress_encoder = ordinal_encoder.fit_transform([df["address"]])

In [40]:
adrress_encoder

array([[0., 0., 0., ..., 0., 0., 0.]])

In [41]:
from sklearn.preprocessing import OneHotEncoder


In [42]:
onehotencoder = OneHotEncoder()

data_transf = onehotencoder.fit_transform([df["area"]])

In [43]:
data_transf.toarray()

array([[1., 1., 1., ..., 1., 1., 1.]])